In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse
import requests
import pprint
import urllib
import os

os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from logger import logger 
from config import cfg
import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
import validate

def get_malware_edr_data(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list = []
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.edr.csv')
        df_list.append( pd.read_csv(filepath))
        print local_path
        
    df = pd.concat(df_list, axis = 0)
    return df

In [2]:
file_list = ["0410", "0411", "0412", "0415", "0416"]
df = get_malware_edr_data(file_list)
df = df.drop_duplicates("url")

/home/sparrow/sparrow/data/sangfor/180410
/home/sparrow/sparrow/data/sangfor/180411
/home/sparrow/sparrow/data/sangfor/180412
/home/sparrow/sparrow/data/sangfor/180415
/home/sparrow/sparrow/data/sangfor/180416


In [3]:
urls = list(df.url)
print "count: %d" % len(df)

count: 22592


In [4]:
url_0 = list(df.loc[df.attr == 0].url)
url_1 = list(df.loc[df.attr == 1].url)
url_2 = list(df.loc[df.attr == 2].url)
url_3 = list(df.loc[df.attr == 3].url)
print "0 malware\t%d\n1 benign\t%d\n2 unknown\t%d\n3 possible benign\t%d" %(len(url_0), len(url_1), len(url_2), len(url_3))
df = df.set_index("url")

0 malware	1707
1 benign	176
2 unknown	20706
3 possible benign	3


### SANGFOR 0301-0402 DOMAIN

In [52]:
root_test_file_path = "/home/sparrow/sparrow/data/edr_test"
download_path = os.path.join(root_test_file_path, "sangfor_0301_0402_domain")
try:
    os.mkdir(download_path)
except Exception as e:
    logger.debug(str(e))

2018-04-20 21:06:30 <ipython-input-52-cba916e16230> [line:6] DEBUG	[Errno 17] File exists: '/home/sparrow/sparrow/data/edr_test/sangfor_0301_0402_domain'


In [53]:
test_set = url_2

predict_malicious_0, predict_dict_detail, predict_dict = regex_domain.malicious_domain_predict(
                      input_file_path = test_set,
                      regex_file_path = "../../data/EXP_sangfor_malware5/regex_publish.txt")
logger.debug("predict_malicious %d/%d" %(len(predict_malicious_0), len(test_set)))

2018-04-20 21:06:31 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_sangfor_malware5/regex_publish.txt


batch index 0	sample index 0	hit domain 5	^19\d{3}\.url\.9xiazaiqi\.com$
batch index 1	sample index 0	hit domain 158	^19\d{3}\.xc\.(:?gongnou|cangpie)\.com$
batch index 2	sample index 0	hit domain 0	^19361\.url\.(:?246546|tudown|7wkw)\.com$
batch index 3	sample index 0	hit domain 41	^19\d{3}\.url\.246546\.com$
batch index 6	sample index 0	hit domain 0	^down050\d{5}\.xdown4\.com$
batch index 4	sample index 0	hit domain 3	^\d{5}\.url\.7wkw\.com$
batch index 7	sample index 0	hit domain 25	^\d{5}\.url\.222bz\.com$
batch index 5	sample index 0	hit domain 259	^\d{5}\.xc\.(:?gongnou|cangpie)\.com$


2018-04-20 21:06:31 <ipython-input-53-1c5255aa427e> [line:6] DEBUG	predict_malicious 1791/20706


In [54]:
print predict_dict.keys()

['^19\\d{3}\\.url\\.9xiazaiqi\\.com$', '^19\\d{3}\\.xc\\.(:?gongnou|cangpie)\\.com$', '^19\\d{3}\\.url\\.246546\\.com$', '^\\d{5}\\.url\\.7wkw\\.com$', '^\\d{5}\\.xc\\.(:?gongnou|cangpie)\\.com$', '^\\d{5}\\.url\\.222bz\\.com$']


In [9]:
validate.download_predict_file(predict_malicious, download_path)

index	0	http://19959.xc.cangpie.com/xiaz/%E5%85%84%E5%BC%9F%E6%BF%80%E5%85%89MFC-7360@311_12607.exe
index	1	http://19962.xc.gongnou.com/xiaz/NCRE%E6%A8%A1%E6%8B%9F%E8%80%83%E8%AF%95%E7%B3%BB%E7%BB%9F%E5%AE%98%E7%BD%91%E4%B8%8B%E8%BD%BD@305_237216.exe
index	2	http://20051.xc.gongnou.com/xiaz/%E7%A0%B4%E4%BA%A7%E5%A7%90%E5%A6%B9%E7%AC%AC%E4%B8%80%E5%AD%A3%E5%9C%A8%E7%BA%BF%E7%9C%8B@610_2.exe
index	3	http://19935.xc.cangpie.com/xiaz/%E7%94%B5%E5%BD%B1%E5%8E%89%E5%AE%B3%E4%BA%86%E6%88%91%E7%9A%84%E5%9B%BD%E9%AB%98%E6%B8%85%E5%AE%8C%E6%95%B4%E7%89%88%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98%E4%B8%8B%E8%BD%BD@1242_2.exe
index	4	http://20029.xc.cangpie.com/xiaz/MySQL%20v5.5.6%20%E7%BB%BF%E8%89%B2%E7%A8%B3%E5%AE%9A%E7%89%88@156_594937.exe
index	5	http://19954.xc.gongnou.com/xiaz/%E6%8B%B3%E7%9A%8798@382_4726.exe
index	6	http://20024.xc.gongnou.com/xiaz/%E7%94%B5%E8%A7%86%E5%89%A7%E5%B0%84%E9%9B%95%E8%8B%B1%E9%9B%84%E4%BC%A0(83%E7%89%88)%E5%85%A8%E9%9B%86@642_2.exe
index	7	http://19963.xc.c

index	59	http://19932.xc.cangpie.com/xiaz/%E8%B6%85%E6%98%9F%E5%AD%A6%E4%B9%A0%E9%80%9A%E7%94%B5%E8%84%91%E7%89%88%E5%AE%98%E6%96%B9%E4%B8%8B%E8%BD%BD@305_197675.exe
index	60	http://19890.xc.cangpie.com/xiaz/%E6%A8%A1%E6%8B%9F%E4%BA%BA%E7%94%9F4%E5%AE%89%E5%8D%93%E7%A0%B4%E8%A7%A3%E7%89%88%20%E7%94%B5%E8%84%91%E7%89%88@58_28741.exe
index	61	http://20032.xc.gongnou.com/xiaz/%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98@584_2.exe
index	62	http://19961.url.246546.com/xiaz/%E9%AC%BC%E7%95%9C%E5%93%A5%E5%93%A5%E8%BD%AF%E8%90%8C%E5%A6%B9%E4%BD%9C%E8%80%85%EF%BC%9A%E6%B5%81%E4%BA%91.txt_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe
index	63	http://20019.xc.cangpie.com/xiaz/%A1%B6%C8%CB%B9%A4%C9%D9%C5%AE3%A1%B7%C3%E2%B0%B2%D7%B0%D3%B2%C5%CC%B0%E6@84_208782.exe
index	64	http://19933.xc.cangpie.com/xiaz/%E5%A4%A9%E7%94%9F%E4%B8%80%E5%AF%B9%E5%85%A8%E9%9B%86%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B%EF%BC%8D%E6%B5%B7%E5%A4%96%E5%89%A7-6080%E6%96%B0%E8%A7%86%E8%A7%89%E5%BD%B1%E9%99%A2@1267

index	114	http://19931.url.246546.com/xiaz/CAD201064%E4%BD%8D%E4%B8%8B%E8%BD%BD%E5%85%8D%E8%B4%B9%E4%B8%AD%E6%96%87%E7%89%88(%E9%99%84%E5%BA%8F%E5%88%97%E5%8F%B7%E5%92%8C%E5%AF%86%E9%92%A5)@317_30716.exe
index	115	http://19929.xc.cangpie.com/down/%E8%A7%86%E9%A2%91%E5%8E%8B%E7%BC%A9%E8%BD%AF%E4%BB%B6%E4%B8%AD%E6%96%87%E7%89%88@133_48387.exe
index	116	http://19905.xc.gongnou.com
index	117	http://19936.xc.gongnou.com/xiaz/%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8%E7%94%B5%E5%BD%B1%E5%85%A8%E9%9B%86%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98@407_2.exe
index	118	http://20026.xc.gongnou.com
index	119	http://20026.xc.gongnou.com/xiaz/%E7%A1%85%E8%B0%B7%E7%AC%AC%E4%BA%94%E5%AD%A3@407_2.exe
index	120	http://19935.url.246546.com/xiaz/%E9%AB%98%E7%AD%89%E6%95%B0%E5%AD%A6%E5%90%8C%E6%B5%8E%E7%AC%AC%E4%B8%83%E7%89%887%E7%89%88%E4%B8%8A%E5%86%8C%E4%B9%A0%E9%A2%98%E5%85%A8%E8%A7%A3%E6%8C%87%E5%8D%97%E8%AF%BE%E5%90%8E%E4%B9%A0%E9%A2%98%E7%AD%94%E6%A1%88%E8%A7%A3%E6%9E%90_20151123185557.pdf_%E7%99%BE%E5%

index	167	http://19931.xc.gongnou.com/xiaz/%E4%B8%8A%E6%B5%B7%E7%A7%91%E6%95%99%E7%89%88%E5%85%AD%E5%B9%B4%E7%BA%A7%E4%B8%8B%E5%86%8C%E5%93%81%E5%BE%B7%E4%B8%8E%E7%A4%BE%E4%BC%9A%E3%80%8A%E7%AC%AC11%E8%AF%BE%E8%AE%A9%E4%B8%96%E7%95%8C%E6%B2%90%E6%B5%B4%E5%92%8C%E5%B9%B3%E7%9A%84%E9%98%B3%E5%85%89%E3%80%8Bppt%E8%AF%BE%E4%BB%B61@483_2.exe
index	168	http://19955.url.246546.com/xiaz/%E6%B0%B4%E8%89%B2%E7%A0%94%E7%A9%B6%E4%BC%9A.pdf_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe
index	169	http://19954.xc.gongnou.com/xiaz/%E8%89%AF%E6%B3%95%E7%94%B5%E8%84%91%E7%89%88%E4%B8%8B%E8%BD%BD@305_333245.exe
index	170	http://20054.xc.gongnou.com
index	171	http://20053.xc.cangpie.com/xiaz/%E6%96%97%E7%A0%B4%E8%8B%8D%E7%A9%B9%E7%AC%AC%E4%BA%8C%E5%AD%A3%E5%85%A8%E9%9B%86_%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98_%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B_%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@439_2.exe
index	172	http://20029.xc.cangpie.com/xiaz/%E7%9B%B2%E5%B1%B1%E9%AB%98%E6%B8%85_%E7%

index	220	http://20051.xc.cangpie.com/xiaz/%E4%B8%8A%E4%BD%8D2%E9%AB%98%E6%B8%85_%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98_%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B_%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@439_2.exe
index	221	http://19604.xc.gongnou.com/xiaz/%E5%94%90%E4%BA%BA%E8%A1%97%E6%8E%A2%E6%A1%882%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@407_2.exe
index	222	http://19967.xc.cangpie.com/down/photoshopcs6%E8%A1%A5%E4%B8%81@134_4038.exe
index	223	http://19423.xc.cangpie.com/xiaz/EpsonL380@311_18890.exe
index	224	http://19934.xc.gongnou.com/xiaz/2015%E6%98%A5%E9%9D%92%E5%B2%9B%E7%89%88%E6%95%B0%E5%AD%A6%E5%85%AB%E4%B8%8B8.2%E3%80%8A%E4%B8%80%E5%85%83%E4%B8%80%E6%AC%A1%E4%B8%8D%E7%AD%89%E5%BC%8F%E3%80%8B%EF%BC%88%E7%AC%AC1%E8%AF%BE%E6%97%B6%EF%BC%89ppt%E8%AF%BE%E4%BB%B6%E4%B8%8B%E8%BD%BD%E9%A1%B5@634_2.exe
index	225	http://19930.xc.gongnou.com/xiaz/IE9%E4%B8%AD%E6%96%87%E7%89%88%E5%AE%98%E6%96%B9%E4%B8%8B%E8%BD%BDwin764%E4%BD%8D@418_29514.exe
index	226	http://19963.url.246546.com/xiaz/%E5%BF%83

...

index	1533	http://19941.xc.gongnou.com/xiaz/%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98@584_2.exe
index	1534	http://19930.url.246546.com/xiaz/%E5%AD%A6%E6%A0%A1%E5%AE%89%E5%85%A8%E6%95%99%E8%82%B2%E5%B9%B3%E5%8F%B0%E7%99%BB%E5%BD%95v2018%E7%BD%91%E9%A1%B5%E7%89%88@316_61484.exe
index	1535	http://19930.xc.cangpie.com/xiaz/flash%E6%8F%92%E4%BB%B6@376_13041.exe
index	1536	http://19334.xc.cangpie.com/xiaz/oledb32r.dll%CF%C2%D4%D8%D2%B3@696_oledb32r.exe
index	1537	http://19960.xc.gongnou.com/xiaz/Microsoft%20Office%202013%20%E5%85%8D%E8%B4%B9%E5%AE%8C%E6%95%B4%E7%89%88%EF%BC%8832&amp;64%E4%BD%8D%EF%BC%89@162_41496.exe
index	1538	http://19935.xc.cangpie.com/xiaz/%E5%9C%B0%E7%90%83%E6%9C%80%E6%9C%AB%E6%97%A5%E9%AB%98%E6%B8%85_%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98_%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B_%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@439_2.exe
index	1539	http://19938.url.246546.com/xiaz/MaduDanRacun.mp3_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe
index	1540	http://

index	1588	http://19960.xc.cangpie.com
index	1589	http://19965.xc.cangpie.com/xiaz/srvinstw.exe@238_248151.exe
index	1590	http://19934.url.246546.com/xiaz/%E7%BE%8E%E7%AF%87%E7%94%B5%E8%84%91%E7%89%88v4.3.3%E5%AE%98%E6%96%B9PC%E7%89%88@136_29867.exe
index	1591	http://19908.xc.cangpie.com
index	1592	http://19931.xc.gongnou.com/xiaz/%E8%B5%B7%E8%B7%91%E7%BA%BF@407_2.exe
index	1593	http://20055.xc.gongnou.com/xiaz/%E9%A3%9E%E8%99%8E%E4%B9%8B%E6%BD%9C%E8%A1%8C%E6%9E%81%E6%88%98-%E7%B2%A4%E8%AF%AD%E7%AC%AC05%E9%9B%86%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B-%E6%B8%AF%E5%89%A7%E7%BD%91@668_2.exe
index	1594	http://19935.url.246546.com
index	1595	http://20053.xc.gongnou.com/xiaz/620.txt@684_2.exe
index	1596	http://20053.xc.gongnou.com/xiaz/%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98@584_2.exe
index	1597	http://20026.xc.gongnou.com/xiaz/DayZ%E5%85%8D%E5%AE%89%E8%A3%85%E7%A1%AC%E7%9B%98%E7%8B%AC%E7%AB%8B%E7%89%88@160_670.exe
index	1598	http://19954.xc.gongnou.com/xiaz/1%E5%AF%B8%E7%85%A7%E7%89%87%E4%BF%AE%E

index	1643	http://19961.xc.gongnou.com/xiaz/%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8%E7%99%BE%E5%BA%A6%E4%BA%91@407_2.exe
index	1644	http://19913.xc.cangpie.com
index	1645	http://20053.xc.cangpie.com/xiaz/%E6%83%85%E9%9A%BE%E8%87%AA%E5%88%B6%E9%AB%98%E6%B8%85_%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98_%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B_%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@439_2.exe
index	1646	http://14614.xc.gongnou.com/down/%E7%8E%8B%E8%80%85%E8%8D%A3%E8%80%80%E7%94%B5%E8%84%91%E7%89%88@626_4967.exe
index	1647	http://19953.xc.gongnou.com/xiaz/2018%E6%BC%AB%E5%A8%81%E7%94%B5%E5%BD%B1%E3%80%8A%E9%BB%91%E8%B1%B9/BlackPanther%E3%80%8B%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@711_2.exe
index	1648	http://19930.url.246546.com/xiaz/VRayforRhino5(%E7%8A%80%E7%89%9BVRay%E6%B8%B2%E6%9F%93%E5%99%A8%E4%B8%8B%E8%BD%BD)v3.40.02%E7%AE%80%E4%BD%93%E4%B8%AD%E6%96%87%E7%89%88@136_37926.exe
index	1649	http://19929.xc.cangpie.com/xiaz/Office2013%E6%BF%80%E6%B4%BB%E5%B7%A5%E5%85%B7@418_29505.exe
index	1650	htt

index	1694	http://20017.xc.gongnou.com
index	1695	http://19936.url.246546.com/xiaz/WPSOffice2018%E4%B8%AA%E4%BA%BA%E7%89%88(wps2018%E5%85%8D%E8%B4%B9%E7%89%88)v10.1.0.7224%E5%AE%98%E6%96%B9%E6%9C%80%E6%96%B0%E7%89%88@317_21036.exe
index	1696	http://20034.xc.cangpie.com/xiaz/%E6%9E%97%E5%B8%88%E5%82%85%E5%9C%A8%E9%A6%96%E5%B0%94%E5%85%A8%E9%9B%86_%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98_%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B_%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@439_2.exe
index	1697	http://20030.xc.gongnou.com/xiaz/%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8,%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8%E7%94%B5%E5%BD%B1,%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B,%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8%E5%85%8D%E8%B4%B9%E8%A7%82%E7%9C%8B,%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8%E9%AB%98%E6%B8%85%E5%AE%8C%E6%95%B4%E7%89%88,%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8%E4%B8%8B%E8%BD%BD@688_2.exe
index	1698	http://19928.xc.gongnou.com/xiaz/zhiwudazhanjiangshiwaigua@249_21356.exe
index	169

index	1752	http://19963.xc.cangpie.com/xiaz/%E6%8D%89%E5%A6%96%E8%AE%B02%E9%AB%98%E6%B8%85%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B-%E7%94%B5%E5%BD%B1%E6%8D%89%E5%A6%96%E8%AE%B02%E4%B8%8B%E8%BD%BD-6080%E6%96%B0%E8%A7%86%E8%A7%89%E5%BD%B1%E9%99%A2@1267_2.exe
index	1753	http://19936.xc.cangpie.com/xiaz/%E5%B9%B8%E7%A6%8F%E4%BA%94%E7%AC%94%E8%BE%93%E5%85%A5%E6%B3%95@271_303518.exe
index	1754	http://20016.xc.gongnou.com/xiaz/%E6%9E%81%E9%99%90%E5%B7%85%E5%B3%B0%E5%AE%98%E6%96%B9%E4%B8%AD%E6%96%87%E7%89%88%E4%B8%8B%E8%BD%BD@723_10104523.exe
index	1755	http://19951.url.246546.com/xiaz/CAD201764%E4%BD%8D%E4%B8%8B%E8%BD%BD%E5%85%8D%E8%B4%B9%E4%B8%AD%E6%96%87%E7%89%88@316_56046.exe
index	1756	http://19929.xc.cangpie.com/xiaz/%E5%A4%A7%E8%AF%9D%E4%B8%89%E5%9B%BD%E4%B9%8B%E6%84%9A%E4%BA%BA%E5%8A%AB%E9%AB%98%E6%B8%85%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B-%E7%94%B5%E5%BD%B1%E5%A4%A7%E8%AF%9D%E4%B8%89%E5%9B%BD%E4%B9%8B%E6%84%9A%E4%BA%BA%E5%8A%AB%E4%B8%8B%E8%BD%BD-6080%E6%96%B0%E8%A7%86%E8%A7%89%E5%BD%B1%E9

In [10]:
reload(validate)
sha256_list = validate.calc_sha256(download_path)
print sha256_list

Counter({'3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff': 1602, 'd540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c': 189})


In [3]:
validate.validate_with_vt_file_report(['d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c',
 '3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff'])

2018-04-20 19:27:50 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-20 19:27:51 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c HTTP/1.1" 200 None
2018-04-20 19:27:51 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com


0 d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c 49


2018-04-20 19:27:52 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff HTTP/1.1" 200 None


1 3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff 49


,sha256,vt
0,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,49
1,3954ed23ba188e657959509fd418273f83f7fa186dbda1...,49


### SANGFOR 0410-0416 DOMAIN

In [55]:
root_test_file_path = "/home/sparrow/sparrow/data/edr_test"
download_path = os.path.join(root_test_file_path, "sangfor_0410_0416_domain")
try:
    os.mkdir(download_path)
except Exception as e:
    logger.debug(str(e))

2018-04-20 21:09:38 <ipython-input-55-b657e9b45d30> [line:6] DEBUG	[Errno 17] File exists: '/home/sparrow/sparrow/data/edr_test/sangfor_0410_0416_domain'


In [56]:
test_set = url_2

predict_malicious_1, predict_dict_detail, predict_dict = regex_domain.malicious_domain_predict(
                      input_file_path = test_set,
                      regex_file_path = "../../data/EXP_sangfor_malware6/regex_publish.txt")
logger.debug("predict_malicious %d/%d" %(len(predict_malicious_1), len(test_set)))

2018-04-20 21:09:39 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_sangfor_malware6/regex_publish.txt


batch index 0	sample index 0	hit domain 0	^\w{2,4}\.renheqk\.com$
batch index 0	sample index 1	hit domain 0	^\w{3,5}\.2018zdfal1\.pw$
batch index 1	sample index 0	hit domain 0	^down\d{5,8}\.xzn56\.com$
batch index 0	sample index 2	hit domain 0	^down0506\d{4}\.(:?xiazaidown|xdown4)\.com$
batch index 1	sample index 1	hit domain 6	^\d{4,5}\.url\.9xiazaiqi\.com$
batch index 1	sample index 2	hit domain 0	^\d{4}\.new\.favo\.uuddu\.com$
batch index 3	sample index 0	hit domain 0	^\w{2,5}\.oumaxite6\.com$
batch index 2	sample index 0	hit domain 158	^19\d{1,3}\.xc\.(:?gongnou|cangpie)\.com$
batch index 2	sample index 1	hit domain 0	^\w{1,5}\.2018zdfb2\.pw$
batch index 3	sample index 1	hit domain 66	^\d{5,8}\.url\.246546\.com$
batch index 3	sample index 2	hit domain 0	^down1\d{4}\.(:?xzn56|xzm56)\.com$
batch index 2	sample index 2	hit domain 0	^[A-Za-z]{5,12}\.zgjjmsw\.com$
batch index 4	sample index 0	hit domain 0	^down1\d{4}\.(:?xzm56|tkg5)\.com$
batch index 4	sample index 1	hit domain 0	^jsapi

2018-04-20 21:09:40 <ipython-input-56-e1d17022db51> [line:6] DEBUG	predict_malicious 1826/20706


In [57]:
predict_dict.keys()

['^\\d{4,5}\\.url\\.9xiazaiqi\\.com$',
 '^\\d{5,8}\\.url\\.246546\\.com$',
 '^19\\d{1,3}\\.xc\\.(:?gongnou|cangpie)\\.com$',
 '^\\d{4,5}\\.xc\\.(:?gongnou|cangpie)\\.com$',
 '^\\d{5}\\.url\\.7wkw\\.com$',
 '^\\d{5}\\.url\\.222bz\\.com$',
 '^\\w{4,6}\\.downyouxi\\.com$',
 '^\\w{3,8}\\.52zsoft\\.com$']

In [9]:
pre_add = list(set(predict_malicious_1) - set(predict_malicious_0))

In [22]:
reload(validate)
df = validate.download_predict_file(pre_add, download_path)

index	0	http://20028.url.246546.com/xiaz/%E5%88%98%E6%83%9C%E5%90%9B-%E8%8F%A9%E6%8F%90%E5%81%88.mp3%E7%8B%AC%E5%AD%A4%E5%A4%A9%E4%B8%8B%E5%8E%9F%E5%A3%B0%E5%B8%A6_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	1	http://20034.url.9xiazaiqi.com/xiaz/%E9%87%91%E5%B1%B1%E5%90%8C%E5%9F%8E%E6%B8%B8%E6%88%8F%E5%A4%A7%E5%8E%85%E6%9C%80%E6%96%B0%E7%89%88@464_2.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	2	http://20062.url.246546.com/xiaz/%E3%80%90yin%E4%B9%B1%E5%B0%8F%E9%95%87%E3%80%91%E5%A2%A8%E5%AF%92%E7%A0%9A%E6%9B%B4%E8%87%B3335.txt%E6%83%85~_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	3	http://20039.url.246546.com/xiaz/%E8%B6%A3%E4%B8%BB%E6%92%AD%E9%BB%84%E9%B3%9D%E9%97%A8%E9%BA%BB%E6%A4%92%E7%90%AA%E7%90%AA%E5%90%8E%E7%BB%AD%E9%BB%84%E9%B3%9D%E6%88%B4%E5%A5%97%E5%A1%9E%E9

index	28	http://20053.url.246546.com/xiaz/%E7%A5%9E%E7%BB%8F%E7%97%87-%E8%AE%B8%E5%8F%88%E6%96%B0.pdf_%E5%BE%AE%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	29	http://20052.url.246546.com/xiaz/%E6%9E%97%E4%BF%8A%E6%9D%B0-%E9%86%89%E8%B5%A4%E5%A3%81.mp3%E6%9E%97%E4%BF%8A%E6%9D%B0_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007%40554_2.ex	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	30	http://20025.url.246546.com/xiaz/AlanWalker-Faded(SaraFarellCover).mp3_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	31	http://dianxin8.52zsoft.com/ghostwjllq.exe	dfcd0a6a7d64af3c3b53b4f13b3541aa6c4b4ca99d0f05b7e1784c4057253a3d
index	32	http://20030.url.246546.com/xiaz/%E4%B8%80%E7%82%B9%E7%82%B9-%E5%91%A8%E6%9D%B0%E4%BC%A6.mp3_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d1398f2

In [24]:
df = df.loc[df.sha256 != "d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c"]
df

,path,sha256,url
11,/home/sparrow/sparrow/data/edr_test/sangfor_04...,9fd52afe8dfb8058fa9567bf7b9ae1f62e82388e24786b...,http://wt40.52zsoft.com/bluestacks.exe
20,/home/sparrow/sparrow/data/edr_test/sangfor_04...,0ed4c224f81c1408c8fbcab767413defd6272283041bb4...,http://down13.downyouxi.com/jiejimingjiangjiaq...
21,/home/sparrow/sparrow/data/edr_test/sangfor_04...,34a15ec672c7147eea377168677c94211d0dede2887983...,http://down3.downyouxi.com/xuerenxiongdi.exe
31,/home/sparrow/sparrow/data/edr_test/sangfor_04...,dfcd0a6a7d64af3c3b53b4f13b3541aa6c4b4ca99d0f05...,http://dianxin8.52zsoft.com/ghostwjllq.exe


In [25]:
validate.validate_with_vt_file_report(
    ['dfcd0a6a7d64af3c3b53b4f13b3541aa6c4b4ca99d0f05b7e1784c4057253a3d',
     '34a15ec672c7147eea377168677c94211d0dede28879830c379523ee76751081',
     '0ed4c224f81c1408c8fbcab767413defd6272283041bb405fdbc0c4e86be7a49',
     '9fd52afe8dfb8058fa9567bf7b9ae1f62e82388e24786bce7a85570eb2b850e3'])

2018-04-20 20:01:12 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-20 20:01:13 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=dfcd0a6a7d64af3c3b53b4f13b3541aa6c4b4ca99d0f05b7e1784c4057253a3d HTTP/1.1" 200 None
2018-04-20 20:01:13 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com


0 dfcd0a6a7d64af3c3b53b4f13b3541aa6c4b4ca99d0f05b7e1784c4057253a3d 0


2018-04-20 20:01:14 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=34a15ec672c7147eea377168677c94211d0dede28879830c379523ee76751081 HTTP/1.1" 200 None
2018-04-20 20:01:14 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com


1 34a15ec672c7147eea377168677c94211d0dede28879830c379523ee76751081 44


2018-04-20 20:01:15 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=0ed4c224f81c1408c8fbcab767413defd6272283041bb405fdbc0c4e86be7a49 HTTP/1.1" 200 None
2018-04-20 20:01:15 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com


2 0ed4c224f81c1408c8fbcab767413defd6272283041bb405fdbc0c4e86be7a49 49


2018-04-20 20:01:16 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=9fd52afe8dfb8058fa9567bf7b9ae1f62e82388e24786bce7a85570eb2b850e3 HTTP/1.1" 200 None


3 9fd52afe8dfb8058fa9567bf7b9ae1f62e82388e24786bce7a85570eb2b850e3 1


,sha256,vt
0,dfcd0a6a7d64af3c3b53b4f13b3541aa6c4b4ca99d0f05...,0
1,34a15ec672c7147eea377168677c94211d0dede2887983...,44
2,0ed4c224f81c1408c8fbcab767413defd6272283041bb4...,49
3,9fd52afe8dfb8058fa9567bf7b9ae1f62e82388e24786b...,1


### VT 0411-0416 DOMAIN - exe

In [62]:
root_test_file_path = "/home/sparrow/sparrow/data/edr_test"
download_path = os.path.join(root_test_file_path, "vt_0411_0416_domain_exe")
try:
    os.mkdir(download_path)
except Exception as e:
    logger.debug(str(e))

2018-04-20 21:13:08 <ipython-input-62-9507d6e9674f> [line:6] DEBUG	[Errno 17] File exists: '/home/sparrow/sparrow/data/edr_test/vt_0411_0416_domain_exe'


In [60]:
test_set = url_2

predict_malicious_2, predict_dict_detail, predict_dict = regex_domain.malicious_domain_predict(
                      input_file_path = test_set,
                      regex_file_path = "../../data/EXP_VirusTotal4/regex_publish.txt")
logger.debug("predict_malicious %d/%d" %(len(predict_malicious_2), len(test_set)))

2018-04-20 21:11:48 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal4/regex_publish.txt


batch index 0	sample index 0	hit domain 0	^2014101\d{10}\.qqldp\.com$
batch index 0	sample index 1	hit domain 0	^\d{1,2}\.duote\.org$
batch index 0	sample index 2	hit domain 0	^(:?i1|www|ww17)\.(:?storebox3|storebox1)\.info$
batch index 1	sample index 0	hit domain 0	^down\.(:?rcdfh|kjdpx|yxdfc|pc876)\.com$
batch index 0	sample index 3	hit domain 0	^(:?dl2|dl5)\.(:?vid4installer|vic6installer|v47installer)\.com$
batch index 0	sample index 4	hit domain 0	^(:?dls1|dls3|dls2)\.(:?aqwitan|afdatin)\.ru$
batch index 1	sample index 1	hit domain 0	^dls3\.aparnis\.ru$
batch index 0	sample index 5	hit domain 0	^(:?a4938164|b2811a66)\.webantivirusprot\.pl$
batch index 1	sample index 2	hit domain 0	^(:?dls4|dls1|dls3|dls2)\.igxeweb\.ru$
batch index 1	sample index 3	hit domain 0	^141630\d{4}\.qiushibaikeba\.com$
batch index 0	sample index 6	hit domain 0	^update\.[A-Za-z]{0,1}in[A-Za-z]{7}\.co\.kr$
batch index 1	sample index 4	hit domain 0	^dls1\.dymkilo\.ru$
batch index 0	sample index 7	hit domain 3

batch index 5	sample index 8	hit domain 0	^(:?download|download0|download14)\.cdn\.(:?fuzezip|jzip)\.com$
batch index 6	sample index 5	hit domain 0	^\w{2}\.\w{2}\.xsj08\.com$
batch index 1	sample index 20	hit domain 0	^(:?okksoft|owosoft|gmdsoft|osdsoft)\.com$
batch index 3	sample index 12	hit domain 0	^xiazai5\.869v\.com$
batch index 4	sample index 13	hit domain 0	^www\.(:?downloadfileshere|download-boosters|download-servers)\.com$
batch index 5	sample index 9	hit domain 0	^(:?3inmh|06u3j|8ilpl9|fauff)\.jinniu\.info$
batch index 3	sample index 13	hit domain 0	^[A-Za-z]{8}\.fb-skandal-video\.de$
batch index 6	sample index 6	hit domain 0	^(:?test|vod|bobo|bin)\.zhyjyj\.com$
batch index 4	sample index 14	hit domain 0	^(:?x|o|dx|cndx)\.uzzf\.com$
batch index 6	sample index 7	hit domain 0	^www\.(:?newqite|pepitu|ntpick)\.com$
batch index 3	sample index 14	hit domain 0	^(:?links-spectr|linkspectr|linksspectr|link-spectr)\.ru$
batch index 5	sample index 10	hit domain 0	^(:?ygnopsi|yderaba|yr

2018-04-20 21:11:49 <ipython-input-60-e22e77e19dd0> [line:6] DEBUG	predict_malicious 44/20706


In [35]:
pre_add = list(set(predict_malicious_2) - set(predict_malicious_1))
len(pre_add)

0

In [61]:
predict_dict.keys()

['^\\w{4,5}\\.downyouxi\\.com$',
 '^\\d{5}\\.url\\.9xiazaiqi\\.com$',
 '^\\d{5}\\.url\\.7wkw\\.com$',
 '^\\d{5}\\.url\\.222bz\\.com$']

In [64]:
validate.download_predict_file( predict_malicious_2, download_path)

index	0	http://19932.url.7wkw.com/xiaz/%E8%B6%85%E5%A3%B0%E9%9A%8F%E5%BF%83%E6%89%80%E6%AC%B2XP%E7%89%88.rar@229_375597.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	1	http://19930.url.222bz.com	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	2	http://19930.url.222bz.com/xiaz/%E4%B8%87%E8%83%BD%E6%97%A0%E7%BA%BF%E7%BD%91%E5%8D%A1%E9%A9%B1%E5%8A%A8forwin764bitv2018.01.24.1733@719_271552.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	3	http://19953.url.7wkw.com/down/%E4%B8%87%E8%83%BDu%E7%9B%98%E9%87%8F%E4%BA%A7%E5%B7%A5%E5%85%B7_1.55%E5%85%8D%E8%B4%B9%E7%BB%BF%E8%89%B2%E7%89%88@50_99942.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	4	http://19954.url.222bz.com/xiaz/%E8%80%81%E6%9D%BF%E9%94%AE-%E6%97%A0%E5%BF%A7%E9%9A%90%E8%97%8F@373_72248.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	5	http://20042.url.222bz.com/xiaz/VLCMediaPlayer%E5%AE%98%E6%96%B9%E

index	42	http://19938.url.222bz.com/xiaz/Proteus8.6@212_222574.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
index	43	http://19957.url.222bz.com/xiaz/HP%BB%DD%C6%D5LaserJetM1005MFP%B6%E0%B9%A6%C4%DC%D2%BB%CC%E5%BB%FA%BC%B4%B2%E5%BC%B4%D3%C3%C7%FD%B6%AF20070507%B0%E6ForWinXP-64@719_107550.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c


,path,sha256,url
0,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://19932.url.7wkw.com/xiaz/%E8%B6%85%E5%A3...
1,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://19930.url.222bz.com
2,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://19930.url.222bz.com/xiaz/%E4%B8%87%E8%8...
3,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://19953.url.7wkw.com/down/%E4%B8%87%E8%83...
4,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://19954.url.222bz.com/xiaz/%E8%80%81%E6%9...
5,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://20042.url.222bz.com/xiaz/VLCMediaPlayer...
6,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://20034.url.9xiazaiqi.com/xiaz/%E9%87%91%...
7,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://19953.url.222bz.com/xiaz/XLLEX.DLL@373_...
8,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://15256.url.222bz.com/xiaz/goldwave@151_3...
9,/home/sparrow/sparrow/data/edr_test/vt_0411_04...,d540e7934d1398f2b6b19e7b7c17e12c387866386f555b...,http://19940.url.222bz.com/xiaz/%E4%B8%87%E5%B...


### VT 0411-0416 DOMAIN - Nonexe

In [37]:
root_test_file_path = "/home/sparrow/sparrow/data/edr_test"
download_path = os.path.join(root_test_file_path, "vt_0411_0416_domain_nonexe")
try:
    os.mkdir(download_path)
except Exception as e:
    logger.debug(str(e))

2018-04-20 20:09:37 <ipython-input-37-7ece346b778c> [line:6] DEBUG	[Errno 17] File exists: '/home/sparrow/sparrow/data/edr_test/vt_0411_0416_domain_nonexe'


In [41]:
test_set = url_2

predict_malicious_3, predict_dict_detail, predict_dict = regex_domain.malicious_domain_predict(
                      input_file_path = test_set,
                      regex_file_path = "../../data/EXP_VirusTotal5/regex_publish.txt")
logger.debug("predict_malicious %d/%d" %(len(predict_malicious_1), len(test_set)))

2018-04-20 20:10:19 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal5/regex_publish.txt


batch index 0	sample index 0	hit domain 0	^click4bestandsafeupdates\.(:?club|win)$
batch index 0	sample index 1	hit domain 0	^(:?yourtrulygreatforupgrade|yourtrulygreatforupgrades)\.download$
batch index 0	sample index 2	hit domain 0	^(:?perfect-codeforfreedeveloped4you|perfectcodeforfreedeveloped4you)\.(:?info|pro|top)$
batch index 1	sample index 0	hit domain 0	^allbetterserviceupgradeclear\.review$
batch index 0	sample index 3	hit domain 0	^\d{1,2}\.duote\.org$
batch index 0	sample index 5	hit domain 0	^\d{3}\.555169\.com$
batch index 1	sample index 1	hit domain 0	^\d{5}\.k9dns\.com$
batch index 2	sample index 0	hit domain 0	^edu\.cn\.byeoo\.cn$
batch index 0	sample index 4	hit domain 0	^www\.[A-Za-z]{2,3}\.com\.pe$
batch index 1	sample index 2	hit domain 0	^edu\.cn\.rosdd\.cn$
batch index 1	sample index 3	hit domain 0	^thecentersofttoupgrade\.(:?bid|trade)$
batch index 1	sample index 4	hit domain 0	^(:?getyourupdatesthatyouwilllovesafe|getyourupdatesthatyouwilllove)\.(:?top|pw)$
bat

batch index 5	sample index 3	hit domain 0	^tlbb\.dfztci\.15525\.www\.gtiuo\.win$
batch index 3	sample index 19	hit domain 0	^edu\.cn\.htrbs\.cn$
batch index 4	sample index 8	hit domain 0	^\w{3,5}\.\w{5,6}\.\w{3,6}\.\w{4,6}\.46utso\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 0	sample index 24	hit domain 0	^paypal\.co\.uk\.\d{4}\.(:?secure-191|secure-911)\.mx$
batch index 1	sample index 18	hit domain 0	^www\.(:?goodplace-contentcentralnew|goodplace-contentcentralall)\.(:?date|bid)$
batch index 3	sample index 20	hit domain 0	^www\.themainplacesetnowforcontentsfree\.(:?win|bid)$
batch index 0	sample index 25	hit domain 0	^www\.(:?greatandonlygoodplacecontentsafeup|greatandonlygoodplacecontentingup)\.(:?date|trade)$
batch index 4	sample index 9	hit domain 0	^wt\d{2,3}\.downyouxi\.com$
batch index 1	sample index 19	hit domain 0	^www\.(:?cr-mufg-card-jp|cr-mufg--card-jp)\.xyz$
batch index 5	sample index 4	hit domain 0	^pm-engineering\.com\.(:?au|com)$
bat

batch index 7	sample index 2	hit domain 0	^z0415p\.eykylw\.tz6b1v\.amqqgo\.velkb3\.17903\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 4	sample index 19	hit domain 0	^[A-Za-z]{3,6}\.nhklg\.cn$
batch index 2	sample index 28	hit domain 0	^thebiggestupdatingversion\.(:?date|review)$
batch index 0	sample index 39	hit domain 0	^nowupgrade24freeforsystemsetpcandmac\.bid$
batch index 3	sample index 33	hit domain 0	^centerallgreatforupgrading\.date$
batch index 4	sample index 20	hit domain 0	^[A-Za-z]{0,1}ass\w{4,5}\.beget\.tech$
batch index 7	sample index 3	hit domain 0	^\d{2,3}\.\d{1,3}\.81\.118\.adsl-pool\.sx\.cn$
batch index 5	sample index 15	hit domain 0	^ask24updateultimateformacandpc\.(:?win|top)$
batch index 1	sample index 25	hit domain 0	^perfectserviceupdating\.(:?date|trade)$
batch index 0	sample index 40	hit domain 0	^www\.yourbiggestbuddyforcontentingall\.download$
batch index 7	sample index 4	hit domain 0	^[A-Za-z]{6,8}\.31dn8uz\.com$
batch index 5	sample in

batch index 6	sample index 17	hit domain 0	^yourperferctprepared4updates\.(:?win|bid|top)$
batch index 3	sample index 45	hit domain 0	^www\.(:?yourmainsourceforcontents|yourmainsourceforcontent)\.(:?win|bid)$
batch index 5	sample index 24	hit domain 0	^(:?goodbroadserviceforupdatingall|goodbroadserviceforupdatesall)\.(:?date|win|review)$
batch index 0	sample index 52	hit domain 0	^clickforsafeready2updatepcandmac\.top$
batch index 0	sample index 53	hit domain 0	^\w{2,6}\.\w{4,6}\.15525\.www\.gtiuo\.win$
batch index 7	sample index 14	hit domain 0	^\w{6}\.mumy47iqd\.com$
batch index 2	sample index 37	hit domain 0	^free\w{4,5}\.beget\.tech$
batch index 6	sample index 18	hit domain 0	^noteupgrade\.softfreeandgreatupdatesloadthisweek\.review$
batch index 4	sample index 30	hit domain 0	^24check4updatesfree4systemsmacandpc\.download$
batch index 1	sample index 36	hit domain 0	^thebigsafegoodstocontentsafe\.trade$
batch index 3	sample index 46	hit domain 0	^clickforsafeandbest4update\.(:?info|

...

batch index 5	sample index 75	hit domain 0	^[^\.]{4,8}unblock\.000webhostapp\.com$
batch index 6	sample index 67	hit domain 0	^\w{6}\.\d{5}\.15525\.www\.gtiuo\.win$
batch index 2	sample index 97	hit domain 0	^legalsoftultimateformacandpc\.bid$
batch index 4	sample index 82	hit domain 0	^netappprepareforpcandmac\.download$
batch index 0	sample index 108	hit domain 0	^\w{4,6}\.\w{6}\.15525\.www\.gtiuo\.win$
batch index 3	sample index 103	hit domain 0	^(:?greatinsthatyouwilllovealways|greatinsthatyouwilllovetoday|greatinsthatyouwilllovenow)\.(:?info|bid|top)$
batch index 1	sample index 85	hit domain 0	^1lira\.(:?3gool|blogspot)\.(:?blogspot|com)\.es$
batch index 7	sample index 69	hit domain 0	^(:?nowinstallupgradefreeforsystemspcandmac|nowinstallupgradefreeforsystemssetpcandmac|nowinstallupgradefreeforsystemspcandmacs)\.win$
batch index 2	sample index 98	hit domain 0	^50yakf\.6244\.phkbg5\.wwdfpd\.37089\.3chv68\.gias81\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 5	

batch index 4	sample index 92	hit domain 0	^\w{3}\.lfhfp\.cn$
batch index 2	sample index 106	hit domain 0	^\w{5,6}\.6x2bzj\.zoy5zy\.43762\.6e2x4j\.30w5am\.bn797d\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 1	sample index 95	hit domain 0	^gov\.cn\.almuu\.cn$
batch index 0	sample index 120	hit domain 0	^er\d{5,6}\.letthemeatcakism\.ru$
batch index 7	sample index 81	hit domain 0	^theultimateandsafe2updates\.(:?bid|site)$
batch index 6	sample index 80	hit domain 0	^91042\.mtlqdw\.f51axv\.\w{6}\.85654\.42829\.c27eg8\.s1ave1\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 4	sample index 93	hit domain 0	^edu\.cn\.lktfc\.cn$
batch index 5	sample index 85	hit domain 0	^exclusiveservice4upgrades\.trade$
batch index 0	sample index 121	hit domain 0	^newsoftready25161456\.info$
batch index 1	sample index 96	hit domain 0	^thebiggestbuddycontentingup\.bid$
batch index 6	sample index 81	hit domain 0	^58643\.\w{2,6}\.87390\.22305\.ytbyzp\.dllm3\.www\.q3537\.d

batch index 2	sample index 118	hit domain 0	^(:?alwaysforevernewestandfastdevelopedsofts|alwaysforevernewestandfastdevelopedsoft)\.(:?win|pw)$
batch index 6	sample index 92	hit domain 0	^user5\d{4}\.vs\.speednames\.com$
batch index 5	sample index 97	hit domain 0	^\d{4}\.verifizieren-sie-ihren-account\.(:?paypal-kundensicherheit|paypal-kundencenter|paypal-kundenzentrale)\.(:?org|com)$
batch index 7	sample index 94	hit domain 0	^er\d{5,6}\.apisbeeteroid\.ru$
batch index 4	sample index 105	hit domain 0	^clicksafeandfastupgrades\.download$
batch index 2	sample index 119	hit domain 0	^(:?allfreeforeverupdate|allfreeforeverupdating)\.(:?date|trade)$
batch index 5	sample index 98	hit domain 0	^\w{3}\.tkycj\.cn$
batch index 6	sample index 93	hit domain 0	^(:?prepareforbiggestmediaupdate|prepareforbiggestmediaupdating|prepareforbiggestupdating)\.(:?win|top)$
batch index 4	sample index 106	hit domain 0	^www\.bankofamerica\.com\.(:?onlinebankingid4761636724|onlinebankingid46722176|onlinebankingid

batch index 5	sample index 111	hit domain 0	^[A-Za-z]{6}\.best\.lt\.ua$
batch index 1	sample index 121	hit domain 0	^\w{5,6}\.nb\.host-domain-lookup\.com$
batch index 4	sample index 118	hit domain 0	^gov\.whlxv\.cn$
batch index 7	sample index 111	hit domain 0	^www\.(:?goodplace-contentcentralnew|goodplace-contentcentralall)\.(:?date|bid|trade)$
batch index 5	sample index 112	hit domain 0	^fabolousuopdatesthatyouwilluse\.(:?info|top)$
batch index 6	sample index 106	hit domain 0	^dns03\.pspvprovalencia\.org$
batch index 4	sample index 119	hit domain 0	^ms7cq4\.di50qa\.0oxb6q\.26k4g1\.68957\.i5lyxv\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 7	sample index 112	hit domain 0	^alwaysdebugforupdating\.(:?bid|top)$
batch index 4	sample index 120	hit domain 0	^thewidevarietycontenting\.trade$
batch index 6	sample index 107	hit domain 0	^(:?bankofamerican-online-access|bankofamerica-onlineaccess-com)\.gq$
batch index 5	sample index 113	hit domain 0	^gov\.cn\.pbkcxp\.cn$
b

2018-04-20 20:10:21 <ipython-input-41-dab1dccb2c75> [line:6] DEBUG	predict_malicious 1826/20706


In [51]:
len(predict_malicious_3)

241

In [42]:
pre_add = list(set(predict_malicious_3) - set(predict_malicious_1))
len(pre_add)

20

In [44]:
reload(validate)
df = validate.download_predict_file(pre_add, download_path)

index	0	http://xz.mqego.com/soft1/cs1.6zuobiqi.zip	0a9bbd241f9c4316c86864da7e772ab87fec7b85af645f521de4fe9106a315ba
index	1	http://c1.mqego.com/soft3/qs.zip	b48d95ac62a854bef5fa759e42335557c03a48c9808362d1a78f76b1d3b7c614
index	2	http://tcy.mqego.com/dilidili.apk	30b81bf6142418faa7277cf6efd87c0912b6c25bbe25d3aea6c746586e29b5e9
index	3	http://dx.mqego.com/soft3/dota%20v6.71b%20ai.zip	e06521aa4264731bc59476dbf79ff8b7fd832e71554abb405f071949c3eaf790
index	4	http://dx.mqego.com/xl/usboot.zip?filename=usboot.zip	b3fd3381034a0e68416590c1cb47e2f5e1754d79edc0e0a45f1050f73a1e1511
index	5	http://xz.mqego.com/soft3/winrar_x86_sc.exe	8e47a0190b5f9e4cc8498a6af5fb5cc61337d31e4d5aec9c27e68cbc749bf09c
index	6	http://xz.mqego.com/soft1/ultraiso_one_gr.zip	a740ba00f06e22fe4b8c66721d12b870e7e3939ed5cad3bd010fd8c359a8ba0f
index	7	http://dx.mqego.com/soft1/fiddler2.zip	827ecdde1f5f39b2150df4ed6a4434c4bc97373ee90d0d803d90ea98b2168152
index	8	http://dl.mqego.com/soft3/gaozhijiazaixiang.zip?filename=gaozhijia

In [48]:
validate.validate_with_vt_file_id(download_path)

2018-04-20 20:51:21 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-20 20:51:23 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-20 20:51:23 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com


0 /home/sparrow/sparrow/data/edr_test/vt_0411_0416_domain_nonexe/1.exe b48d95ac62a854bef5fa759e42335557c03a48c9808362d1a78f76b1d3b7c614


2018-04-20 20:51:32 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-20 20:51:32 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com


1 /home/sparrow/sparrow/data/edr_test/vt_0411_0416_domain_nonexe/3.exe e06521aa4264731bc59476dbf79ff8b7fd832e71554abb405f071949c3eaf790


2018-04-20 20:51:34 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-20 20:51:34 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com


...

2018-04-20 20:52:38 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-20 20:52:38 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com


18 /home/sparrow/sparrow/data/edr_test/vt_0411_0416_domain_nonexe/18.exe 70265966c7d0a5e26a48bfee8f2dea7ae7a696f8cb37dbffb155498b1ca4f4d2


2018-04-20 20:52:41 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580


19 /home/sparrow/sparrow/data/edr_test/vt_0411_0416_domain_nonexe/5.exe 8e47a0190b5f9e4cc8498a6af5fb5cc61337d31e4d5aec9c27e68cbc749bf09c


[u'b48d95ac62a854bef5fa759e42335557c03a48c9808362d1a78f76b1d3b7c614',
 u'e06521aa4264731bc59476dbf79ff8b7fd832e71554abb405f071949c3eaf790',
 u'ba76ef5d4e1294882ab2065fd61de2799d9af23a1aa14544bef756e7a611da87',
 u'942904c1dd15b0cf86118326b9f077ab9d7ac0605a49f613213cee1fcba95bb8',
 u'61481b464d7012d07d6ed38d82a6cd0fa3a00e0c1d01a188aac6ea21f547c773',
 u'7ab9fecbebb8c35cec7aac57de9577211f13090c415ba1c02d27cbfdb1791191',
 u'6c036b8bb0fa920bba4312baaa69017ada40945eb1d26aec8951e3df290db792',
 u'30b81bf6142418faa7277cf6efd87c0912b6c25bbe25d3aea6c746586e29b5e9',
 u'827ecdde1f5f39b2150df4ed6a4434c4bc97373ee90d0d803d90ea98b2168152',
 u'2c11a9996f58ddb150ab4a8b5218c9e299c4481d4ac404fe3d73a1153bad1c81',
 u'b3fd3381034a0e68416590c1cb47e2f5e1754d79edc0e0a45f1050f73a1e1511',
 u'8cb55bf22b2b6d55da45f73d985d7067ff34ba38e8372046e0637f9fb21e0337',
 u'aa8e04cd4cf56392c2301154565f273802130d1362c833dc167f7cec815cb679',
 u'26835cee3190a98bf5f96fd7ff58bb1af61fdb7c6856a4ab3bf6390c51c0922b',
 u'a740ba00f06e22fe4